In [2]:
!/opt/conda/envs/rapids/bin/python -m pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 106.5 MB/s eta 0:00:0000:0100:01
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 77.3 MB/s eta 0:00:00
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 138.4 MB/s eta 0:00:00
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl (906.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 64.1 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)


In [3]:
import torch
import torch.nn as nn
import pandas as pd

# Reload the file to confirm the embeddings
cls_1 = pd.read_csv('cls_embeddings 2.csv')
cls_2 = pd.read_csv('patient_embeddings(2).csv')



In [10]:
cls_1.columns = cls_1.columns.str.lower()
cls_2.columns = cls_2.columns.str.lower()

In [11]:
merged_on_patient_id = pd.merge(cls_1, cls_2, on='patient_id', how='inner')

In [13]:
# Assume the CLS embeddings are stored as numerical columns in the file
cls_embeddings = merged_on_patient_id.values  # Convert to NumPy array
cls_tensor = torch.tensor(cls_embeddings, dtype=torch.float32)  # Convert to tensor

# Define the Feed-Forward Neural Network
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the network
input_dim = cls_tensor.shape[1]  # Number of features in the CLS embeddings
hidden_dim = 512  # Hidden layer size
output_dim = 10  # Number of output classes
model = FeedForwardNN(input_dim, hidden_dim, output_dim)

# Perform a forward pass on the CLS embeddings
output = model(cls_tensor)

# Display the network architecture and sample output
model, output[:5]  # Display first 5 predictions for testing purposes


(FeedForwardNN(
   (fc1): Linear(in_features=1537, out_features=512, bias=True)
   (relu): ReLU()
   (fc2): Linear(in_features=512, out_features=10, bias=True)
 ),
 tensor([[  92.2518,   -4.0654,  -96.9748,  128.2229,   27.2384,   40.5467,
           160.0581,  -88.9057, -123.1837, -129.8535],
         [   1.6930,   -0.3619,   -0.9373,    2.1001,    0.3246,    0.7136,
             2.1164,   -1.3414,   -1.9518,   -1.9935],
         [  60.8977,   -2.7809,  -63.7357,   84.5718,   17.9232,   26.7671,
           105.4081,  -58.5702,  -81.2424,  -85.6398],
         [  73.7007,   -3.2758,  -77.3141,  102.3518,   21.6597,   32.3490,
           127.7330,  -70.9092,  -98.3438, -103.6566],
         [  30.8414,   -1.5491,  -31.8776,   42.7288,    9.0555,   13.5718,
            53.0303,  -29.5533,  -41.0231,  -43.2590]],
        grad_fn=<SliceBackward0>))